# Initialization

## Import Libraries

In [1]:
# data processing
import numpy as np
import pandas as pd 
from collections import defaultdict
import time

# data visualization
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
sns.set()
import urllib.request


import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2021-12-11 18:44:07.381011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 18:44:07.392332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 18:44:07.393102: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Import Custom Module

In [2]:
#adjust filepath so that working directory is folder "recreating-residual-attention-network"
import os

PROJECT_PATH = '/home/ecbm4040/recreating-residual-attention-network'
os.chdir(PROJECT_PATH)

In [ ]:
# If using Google Colab
from google.colab import drive
BASE_PATH = '/content/drive'
drive.mount(BASE_PATH)

# change directory
import os
PROJECT_PATH = os.path.join(BASE_PATH, "MyDrive", "ECBM4040", "FinalProject", "recreating-residual-attention-network")
os.chdir(PROJECT_PATH)

In [3]:
# Import created modules
from src.models.ResidualAttentionNetwork import ResidualAttentionNetwork, Attention56, Attention92, Attention128, Attention164
from src.utils import generate_data

2021-12-11 18:44:07.516299: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-11 18:44:07.516762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 18:44:07.517554: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 18:44:07.518184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

## Custom Callbacks

In [4]:
### Callback to save per epoch time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        # Save epoch time directly on the csv_logger
        logs['per_epoch_time'] = time.time() - self.epoch_time_start

### Callback to stop when certain accuracy is reached
class EarlyStoppingByAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, monitor='accuracy', value=0.98, verbose=0):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

# Modelling CIFAR-10

In [5]:
INPUT_SHAPE, NUM_CLASS, train_ds, val_ds, test_ds, _ = generate_data.get_cifar10()

In [6]:
BATCH_SIZE = 256

In [7]:
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
STEP_SIZE = len(train_ds)

## Naive Attention Learning

In this part, the experiment is done using Naive Attention Learning mechanism

### Attention-56

In [9]:
N_EPOCH = 100
LR = 1e-3

In [10]:
ran_model = Attention56(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             multiple                  864       
                                                                 
 batch_normalization (BatchN  multiple                 128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  multiple                 

In [11]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/1-a56-nal-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/1-a56-nal-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [12]:
a56_nal_model = ran_model

a56_nal_model.compile(optimizer=our_optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(), 
                      metrics=['accuracy'])


history = a56_nal_model.fit(train_ds,
                            validation_data=val_ds,
                            callbacks=callbacks,
                            epochs=N_EPOCH, verbose=1)

Epoch 1/100


2021-12-10 01:24:13.362181: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200


157/157 [==============================] - 34s 124ms/step - loss: 2.4147 - accuracy: 0.1653 - val_loss: 2.8198 - val_accuracy: 0.1000 - per_epoch_time: 34.2333
Epoch 2/100
157/157 [==============================] - 17s 107ms/step - loss: 2.1098 - accuracy: 0.2437 - val_loss: 2.4934 - val_accuracy: 0.1345 - per_epoch_time: 16.8126
Epoch 3/100
157/157 [==============================] - 17s 107ms/step - loss: 1.9906 - accuracy: 0.2894 - val_loss: 2.1168 - val_accuracy: 0.2334 - per_epoch_time: 16.8017
Epoch 4/100
157/157 [==============================] - 17s 106ms/step - loss: 1.9104 - accuracy: 0.3109 - val_loss: 1.9089 - val_accuracy: 0.3151 - per_epoch_time: 16.7080
Epoch 5/100
157/157 [==============================] - 17s 107ms/step - loss: 1.8445 - accuracy: 0.3345 - val_loss: 1.8669 - val_accuracy: 0.3407 - per_epoch_time: 16.7772
Epoch 6/100
157/157 [==============================] - 17s 107ms/step - loss: 1.7991 - accuracy: 0.3466 - val_loss: 2.0127 - val_accuracy: 0.3054 - per_

157/157 [==============================] - 17s 107ms/step - loss: 0.4888 - accuracy: 0.8375 - val_loss: 0.6446 - val_accuracy: 0.7978 - per_epoch_time: 16.7214
Epoch 96/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4873 - accuracy: 0.8392 - val_loss: 0.6624 - val_accuracy: 0.7909 - per_epoch_time: 16.5859
Epoch 97/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4773 - accuracy: 0.8403 - val_loss: 0.6616 - val_accuracy: 0.7918 - per_epoch_time: 16.6485
Epoch 98/100
157/157 [==============================] - 17s 107ms/step - loss: 0.4821 - accuracy: 0.8407 - val_loss: 0.6460 - val_accuracy: 0.7960 - per_epoch_time: 16.8286
Epoch 99/100
157/157 [==============================] - 17s 108ms/step - loss: 0.4777 - accuracy: 0.8423 - val_loss: 0.6440 - val_accuracy: 0.7961 - per_epoch_time: 16.9069
Epoch 100/100
157/157 [==============================] - 17s 107ms/step - loss: 0.4779 - accuracy: 0.8410 - val_loss: 0.6464 - val_accuracy: 0.7958 

### Attention-92

Faster learning rate decay and add gradient clipping in optimizer to avoid exploding gradient

In [9]:
N_EPOCH = 100
LR = 1e-3

In [10]:
ran_model = Attention92(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention92"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             multiple                  864       
                                                                 
 batch_normalization (BatchN  multiple                 128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  multiple                 

In [11]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/2-a92-nal-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/2-a92-nal-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=0.1)

In [12]:
a92_nal_model = ran_model

a92_nal_model.compile(optimizer=our_optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(), 
                      metrics=['accuracy'])


history = a92_nal_model.fit(train_ds,
                            validation_data=val_ds,
                            callbacks=callbacks,
                            epochs=N_EPOCH, verbose=1)

Epoch 1/100


2021-12-11 18:44:57.870504: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200


157/157 [==============================] - 64s 224ms/step - loss: 2.3527 - accuracy: 0.1332 - val_loss: 2.3204 - val_accuracy: 0.1000 - per_epoch_time: 64.2904
Epoch 2/100
157/157 [==============================] - 30s 191ms/step - loss: 2.1155 - accuracy: 0.2019 - val_loss: 2.6214 - val_accuracy: 0.1215 - per_epoch_time: 29.9907
Epoch 3/100
157/157 [==============================] - 30s 194ms/step - loss: 2.0246 - accuracy: 0.2486 - val_loss: 2.0104 - val_accuracy: 0.2202 - per_epoch_time: 30.3704
Epoch 4/100
157/157 [==============================] - 31s 196ms/step - loss: 1.9611 - accuracy: 0.2883 - val_loss: 2.0290 - val_accuracy: 0.2405 - per_epoch_time: 30.7126
Epoch 5/100
157/157 [==============================] - 30s 192ms/step - loss: 1.9610 - accuracy: 0.2898 - val_loss: 2.2491 - val_accuracy: 0.2090 - per_epoch_time: 30.0809
Epoch 6/100
157/157 [==============================] - 30s 193ms/step - loss: 1.8728 - accuracy: 0.3155 - val_loss: 2.0924 - val_accuracy: 0.2587 - per_

157/157 [==============================] - 30s 189ms/step - loss: 0.9515 - accuracy: 0.6690 - val_loss: 1.4294 - val_accuracy: 0.6074 - per_epoch_time: 29.6638
Epoch 49/100
157/157 [==============================] - 29s 185ms/step - loss: 0.9367 - accuracy: 0.6736 - val_loss: 1.1003 - val_accuracy: 0.6211 - per_epoch_time: 29.0926
Epoch 50/100
157/157 [==============================] - 30s 190ms/step - loss: 0.9265 - accuracy: 0.6783 - val_loss: 1.4455 - val_accuracy: 0.5911 - per_epoch_time: 29.8143
Epoch 51/100
157/157 [==============================] - 30s 189ms/step - loss: 0.8172 - accuracy: 0.7163 - val_loss: 0.8107 - val_accuracy: 0.7176 - per_epoch_time: 29.7003
Epoch 52/100
157/157 [==============================] - 30s 189ms/step - loss: 0.7937 - accuracy: 0.7254 - val_loss: 0.8108 - val_accuracy: 0.7183 - per_epoch_time: 29.6511
Epoch 53/100
157/157 [==============================] - 29s 183ms/step - loss: 0.7713 - accuracy: 0.7315 - val_loss: 0.8212 - val_accuracy: 0.7182 -

157/157 [==============================] - 30s 188ms/step - loss: 0.6080 - accuracy: 0.7890 - val_loss: 1.0438 - val_accuracy: 0.6942 - per_epoch_time: 29.5065
Epoch 96/100
157/157 [==============================] - 29s 186ms/step - loss: 0.6146 - accuracy: 0.7872 - val_loss: 0.7329 - val_accuracy: 0.7569 - per_epoch_time: 29.1388
Epoch 97/100
157/157 [==============================] - 29s 186ms/step - loss: 0.6096 - accuracy: 0.7869 - val_loss: 0.7350 - val_accuracy: 0.7585 - per_epoch_time: 29.2250
Epoch 98/100
157/157 [==============================] - 29s 182ms/step - loss: 0.6109 - accuracy: 0.7893 - val_loss: 0.7778 - val_accuracy: 0.7450 - per_epoch_time: 28.6130
Epoch 99/100
157/157 [==============================] - 29s 184ms/step - loss: 0.6093 - accuracy: 0.7905 - val_loss: 0.7197 - val_accuracy: 0.7586 - per_epoch_time: 28.9329
Epoch 100/100
157/157 [==============================] - 29s 182ms/step - loss: 0.6063 - accuracy: 0.7890 - val_loss: 0.7470 - val_accuracy: 0.7548 

### Attention-128

In [34]:
N_EPOCH = 100
LR = 1e-3

In [35]:
ran_model = Attention128(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention128_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_1974 (Conv2D)        multiple                  864       
                                                                 
 batch_normalization_1512 (B  multiple                 128       
 atchNormalization)                                              
                                                                 
 re_lu_1512 (ReLU)           multiple                  0         
                                                                 
 max_pooling2d_102 (MaxPooli  multiple                 0         
 ng2D)                                                           
                                                                 
 residual_unit_468 (Residual  multiple              

In [36]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/3-a128-nal-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/3-a128-nal-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=0.01)

In [37]:
a128_nal_model = ran_model

a128_nal_model.compile(optimizer=our_optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(), 
                      metrics=['accuracy'])


history = a128_nal_model.fit(train_ds,
                            validation_data=val_ds,
                            callbacks=callbacks,
                            epochs=N_EPOCH, verbose=1)

Epoch 1/100
157/157 [==============================] - 85s 307ms/step - loss: 2.3926 - accuracy: 0.1021 - val_loss: 2.3028 - val_accuracy: 0.1000 - per_epoch_time: 85.4572
Epoch 2/100
157/157 [==============================] - 43s 277ms/step - loss: 2.3054 - accuracy: 0.1305 - val_loss: 2.2985 - val_accuracy: 0.1049 - per_epoch_time: 43.4743
Epoch 3/100
157/157 [==============================] - 43s 277ms/step - loss: 2.2066 - accuracy: 0.1778 - val_loss: 2.1799 - val_accuracy: 0.1823 - per_epoch_time: 43.4696
Epoch 4/100
157/157 [==============================] - 43s 271ms/step - loss: 2.1070 - accuracy: 0.2135 - val_loss: 3.4339 - val_accuracy: 0.1993 - per_epoch_time: 42.6064
Epoch 5/100
157/157 [==============================] - 42s 271ms/step - loss: 2.0127 - accuracy: 0.2333 - val_loss: 2.4082 - val_accuracy: 0.1984 - per_epoch_time: 42.4804
Epoch 6/100
157/157 [==============================] - 42s 271ms/step - loss: 1.9471 - accuracy: 0.2462 - val_loss: 2.4416 - val_accuracy: 0

157/157 [==============================] - 42s 268ms/step - loss: 1.0658 - accuracy: 0.6295 - val_loss: 1.0322 - val_accuracy: 0.6526 - per_epoch_time: 42.0753
Epoch 49/100
157/157 [==============================] - 43s 272ms/step - loss: 1.0678 - accuracy: 0.6293 - val_loss: 1.2647 - val_accuracy: 0.5601 - per_epoch_time: 42.6369
Epoch 50/100
157/157 [==============================] - 41s 264ms/step - loss: 1.0546 - accuracy: 0.6356 - val_loss: 1.0076 - val_accuracy: 0.6531 - per_epoch_time: 41.5007
Epoch 51/100
157/157 [==============================] - 41s 262ms/step - loss: 1.0487 - accuracy: 0.6357 - val_loss: 1.3027 - val_accuracy: 0.5919 - per_epoch_time: 41.1499
Epoch 52/100
157/157 [==============================] - 41s 262ms/step - loss: 1.0400 - accuracy: 0.6414 - val_loss: 1.7864 - val_accuracy: 0.4193 - per_epoch_time: 41.1384
Epoch 53/100
157/157 [==============================] - 42s 269ms/step - loss: 1.0314 - accuracy: 0.6429 - val_loss: 1.9610 - val_accuracy: 0.4788 -

157/157 [==============================] - 41s 261ms/step - loss: 0.7445 - accuracy: 0.7481 - val_loss: 0.8207 - val_accuracy: 0.7264 - per_epoch_time: 40.9206
Epoch 96/100
157/157 [==============================] - 41s 263ms/step - loss: 0.7360 - accuracy: 0.7498 - val_loss: 0.8425 - val_accuracy: 0.7270 - per_epoch_time: 41.2451
Epoch 97/100
157/157 [==============================] - 42s 265ms/step - loss: 0.7423 - accuracy: 0.7502 - val_loss: 0.8780 - val_accuracy: 0.7221 - per_epoch_time: 41.6121
Epoch 98/100
157/157 [==============================] - 43s 272ms/step - loss: 0.7327 - accuracy: 0.7511 - val_loss: 7604.2778 - val_accuracy: 0.7118 - per_epoch_time: 42.7256
Epoch 99/100
157/157 [==============================] - 42s 268ms/step - loss: 0.7343 - accuracy: 0.7483 - val_loss: 0.8354 - val_accuracy: 0.7329 - per_epoch_time: 42.0042
Epoch 100/100
157/157 [==============================] - 42s 267ms/step - loss: 0.7308 - accuracy: 0.7518 - val_loss: 0.7901 - val_accuracy: 0.73

### Attention-164

In [ ]:
N_EPOCH = 100
LR = 1e-3

In [ ]:
ran_model = Attention164(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

In [ ]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/4-a192-nal-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/4-a192-nal-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=0.001)

In [ ]:
a192_nal_model = ran_model

a192_nal_model.compile(optimizer=our_optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(), 
                      metrics=['accuracy'])


history = a192_nal_model.fit(train_ds,
                            validation_data=val_ds,
                            callbacks=callbacks,
                            epochs=N_EPOCH, verbose=1)

## Attention-Residual Learning

In this part, the experiment is done using Residual Attention Learning mechanism

### Attention-56

In [13]:
N_EPOCH = 100
LR = 1e-3

In [14]:
ran_model = Attention56(input_shape=INPUT_SHAPE, num_class=NUM_CLASS)
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention56_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_139 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_107 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_107 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 residual_unit_33 (ResidualU  multiple               

In [15]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/1-a56-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/1-a56-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [16]:
a56_model = ran_model

a56_model.compile(optimizer=our_optimizer, 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a56_model.fit(train_ds,
                        validation_data=val_ds,
                        callbacks=callbacks,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/100
157/157 [==============================] - 31s 119ms/step - loss: 2.4594 - accuracy: 0.1515 - val_loss: 2.6538 - val_accuracy: 0.1000 - per_epoch_time: 30.6641
Epoch 2/100
157/157 [==============================] - 17s 107ms/step - loss: 2.0942 - accuracy: 0.2455 - val_loss: 2.5194 - val_accuracy: 0.1320 - per_epoch_time: 16.7516
Epoch 3/100
157/157 [==============================] - 17s 108ms/step - loss: 1.9156 - accuracy: 0.3052 - val_loss: 1.9570 - val_accuracy: 0.2832 - per_epoch_time: 16.8760
Epoch 4/100
157/157 [==============================] - 17s 106ms/step - loss: 1.8190 - accuracy: 0.3408 - val_loss: 1.7478 - val_accuracy: 0.3669 - per_epoch_time: 16.6506
Epoch 5/100
157/157 [==============================] - 17s 107ms/step - loss: 1.7587 - accuracy: 0.3671 - val_loss: 1.9506 - val_accuracy: 0.3231 - per_epoch_time: 16.8248
Epoch 6/100
157/157 [==============================] - 17s 109ms/step - loss: 1.7030 - accuracy: 0.3837 - val_loss: 1.8401 - val_accuracy: 0

157/157 [==============================] - 17s 106ms/step - loss: 0.4640 - accuracy: 0.8471 - val_loss: 0.6167 - val_accuracy: 0.8008 - per_epoch_time: 16.6375
Epoch 96/100
157/157 [==============================] - 17s 107ms/step - loss: 0.4567 - accuracy: 0.8495 - val_loss: 0.6008 - val_accuracy: 0.8041 - per_epoch_time: 16.8036
Epoch 97/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4642 - accuracy: 0.8476 - val_loss: 0.5895 - val_accuracy: 0.8070 - per_epoch_time: 16.7101
Epoch 98/100
157/157 [==============================] - 17s 107ms/step - loss: 0.4549 - accuracy: 0.8497 - val_loss: 0.6205 - val_accuracy: 0.8021 - per_epoch_time: 16.7496
Epoch 99/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4526 - accuracy: 0.8509 - val_loss: 0.6155 - val_accuracy: 0.8012 - per_epoch_time: 16.6782
Epoch 100/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4524 - accuracy: 0.8512 - val_loss: 0.6105 - val_accuracy: 0.8047 

### Attention-92

Decrease initial learning rate and add gradient clipping in optimizer to avoid exploding gradient

In [13]:
N_EPOCH = 100
LR = 1e-3

In [14]:
ran_model = Attention92(input_shape=INPUT_SHAPE, num_class=NUM_CLASS)
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention92_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_253 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_194 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_194 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_13 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_60 (ResidualU  multiple               

In [15]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/2-a92-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/2-a92-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[50*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=0.1)

In [16]:
a92_model = ran_model

a92_model.compile(optimizer=our_optimizer, 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a92_model.fit(train_ds,
                        validation_data=val_ds,
                        callbacks=callbacks,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/100
157/157 [==============================] - 59s 211ms/step - loss: 2.3720 - accuracy: 0.1159 - val_loss: 2.3035 - val_accuracy: 0.1000 - per_epoch_time: 58.7535
Epoch 2/100
157/157 [==============================] - 30s 191ms/step - loss: 2.2853 - accuracy: 0.1468 - val_loss: 2.2242 - val_accuracy: 0.1464 - per_epoch_time: 29.9645
Epoch 3/100
157/157 [==============================] - 30s 188ms/step - loss: 2.1246 - accuracy: 0.1850 - val_loss: 114.2793 - val_accuracy: 0.1097 - per_epoch_time: 29.5266
Epoch 4/100
157/157 [==============================] - 30s 191ms/step - loss: 2.0353 - accuracy: 0.2171 - val_loss: 3.0858 - val_accuracy: 0.1737 - per_epoch_time: 30.0265
Epoch 5/100
157/157 [==============================] - 30s 189ms/step - loss: 1.9260 - accuracy: 0.2539 - val_loss: 2.9661 - val_accuracy: 0.1956 - per_epoch_time: 29.6702
Epoch 6/100
157/157 [==============================] - 30s 191ms/step - loss: 1.8616 - accuracy: 0.2784 - val_loss: 3.2445 - val_accuracy:

157/157 [==============================] - 30s 191ms/step - loss: 1.0240 - accuracy: 0.6489 - val_loss: 1.4898 - val_accuracy: 0.5514 - per_epoch_time: 29.9618
Epoch 49/100
157/157 [==============================] - 30s 189ms/step - loss: 1.0312 - accuracy: 0.6471 - val_loss: 1.3579 - val_accuracy: 0.5844 - per_epoch_time: 29.7048
Epoch 50/100
157/157 [==============================] - 31s 195ms/step - loss: 1.0078 - accuracy: 0.6560 - val_loss: 1.1531 - val_accuracy: 0.6200 - per_epoch_time: 30.6241
Epoch 51/100
157/157 [==============================] - 30s 193ms/step - loss: 0.9085 - accuracy: 0.6901 - val_loss: 0.9034 - val_accuracy: 0.6913 - per_epoch_time: 30.2399
Epoch 52/100
157/157 [==============================] - 30s 193ms/step - loss: 0.8742 - accuracy: 0.7021 - val_loss: 0.8454 - val_accuracy: 0.7089 - per_epoch_time: 30.2042
Epoch 53/100
157/157 [==============================] - 30s 189ms/step - loss: 0.8587 - accuracy: 0.7086 - val_loss: 0.8565 - val_accuracy: 0.7089 -

157/157 [==============================] - 30s 190ms/step - loss: 0.6980 - accuracy: 0.7631 - val_loss: 0.8014 - val_accuracy: 0.7404 - per_epoch_time: 29.8172
Epoch 96/100
157/157 [==============================] - 30s 188ms/step - loss: 0.6984 - accuracy: 0.7651 - val_loss: 0.7333 - val_accuracy: 0.7610 - per_epoch_time: 29.5706
Epoch 97/100
157/157 [==============================] - 29s 184ms/step - loss: 0.7021 - accuracy: 0.7646 - val_loss: 0.7813 - val_accuracy: 0.7422 - per_epoch_time: 28.8469
Epoch 98/100
157/157 [==============================] - 29s 186ms/step - loss: 0.6912 - accuracy: 0.7647 - val_loss: 0.8031 - val_accuracy: 0.7376 - per_epoch_time: 29.2670
Epoch 99/100
157/157 [==============================] - 30s 189ms/step - loss: 0.6858 - accuracy: 0.7669 - val_loss: 0.7728 - val_accuracy: 0.7469 - per_epoch_time: 29.6865
Epoch 100/100
157/157 [==============================] - 30s 191ms/step - loss: 0.6843 - accuracy: 0.7668 - val_loss: 0.7629 - val_accuracy: 0.7530 

### Attention-128

In [38]:
N_EPOCH = 100
LR = 1e-3

In [39]:
ran_model = Attention128(input_shape=INPUT_SHAPE, num_class=NUM_CLASS)
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention128_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_2341 (Conv2D)        multiple                  864       
                                                                 
 batch_normalization_1793 (B  multiple                 128       
 atchNormalization)                                              
                                                                 
 re_lu_1793 (ReLU)           multiple                  0         
                                                                 
 max_pooling2d_121 (MaxPooli  multiple                 0         
 ng2D)                                                           
                                                                 
 residual_unit_555 (Residual  multiple              

In [40]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/3-a128-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/3-a128-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=0.01)

In [41]:
a128_model = ran_model

a128_model.compile(optimizer=our_optimizer, 
                   loss=tf.keras.losses.CategoricalCrossentropy(), 
                   metrics=['accuracy'])


history = a128_model.fit(train_ds,
                         validation_data=val_ds,
                         callbacks=callbacks,
                         epochs=N_EPOCH, verbose=1)

Epoch 1/100
157/157 [==============================] - 86s 304ms/step - loss: 2.3707 - accuracy: 0.1129 - val_loss: 2.3033 - val_accuracy: 0.1000 - per_epoch_time: 86.1712
Epoch 2/100
157/157 [==============================] - 43s 274ms/step - loss: 2.2454 - accuracy: 0.1642 - val_loss: 6.9415 - val_accuracy: 0.1000 - per_epoch_time: 42.9939
Epoch 3/100
157/157 [==============================] - 42s 269ms/step - loss: 2.1108 - accuracy: 0.1936 - val_loss: 2.0375 - val_accuracy: 0.1901 - per_epoch_time: 42.2407
Epoch 4/100
157/157 [==============================] - 42s 270ms/step - loss: 2.0017 - accuracy: 0.2298 - val_loss: 3.2848 - val_accuracy: 0.2005 - per_epoch_time: 42.3764
Epoch 5/100
157/157 [==============================] - 43s 276ms/step - loss: 1.9363 - accuracy: 0.2401 - val_loss: 1.9300 - val_accuracy: 0.2276 - per_epoch_time: 43.3318
Epoch 6/100
157/157 [==============================] - 43s 272ms/step - loss: 1.8931 - accuracy: 0.2498 - val_loss: 2.8010 - val_accuracy: 0

157/157 [==============================] - 43s 272ms/step - loss: 1.0329 - accuracy: 0.6474 - val_loss: 1.1586 - val_accuracy: 0.5984 - per_epoch_time: 42.7394
Epoch 49/100
157/157 [==============================] - 42s 269ms/step - loss: 1.0272 - accuracy: 0.6510 - val_loss: 10.1989 - val_accuracy: 0.2128 - per_epoch_time: 42.2303
Epoch 50/100
157/157 [==============================] - 43s 271ms/step - loss: 1.0110 - accuracy: 0.6585 - val_loss: 4.4571 - val_accuracy: 0.3286 - per_epoch_time: 42.5669
Epoch 51/100
157/157 [==============================] - 41s 264ms/step - loss: 1.0098 - accuracy: 0.6562 - val_loss: 1.3697 - val_accuracy: 0.5980 - per_epoch_time: 41.4880
Epoch 52/100
157/157 [==============================] - 42s 270ms/step - loss: 1.0027 - accuracy: 0.6622 - val_loss: 1.4163 - val_accuracy: 0.5075 - per_epoch_time: 42.3729
Epoch 53/100
157/157 [==============================] - 43s 272ms/step - loss: 0.9982 - accuracy: 0.6626 - val_loss: 1.2497 - val_accuracy: 0.5707 

157/157 [==============================] - 43s 271ms/step - loss: 0.6785 - accuracy: 0.7767 - val_loss: 0.8278 - val_accuracy: 0.7303 - per_epoch_time: 42.5526
Epoch 96/100
157/157 [==============================] - 42s 266ms/step - loss: 0.6759 - accuracy: 0.7765 - val_loss: 0.7636 - val_accuracy: 0.7595 - per_epoch_time: 41.6832
Epoch 97/100
157/157 [==============================] - 43s 274ms/step - loss: 0.6750 - accuracy: 0.7771 - val_loss: 0.7795 - val_accuracy: 0.7504 - per_epoch_time: 43.0087
Epoch 98/100
157/157 [==============================] - 43s 275ms/step - loss: 0.6742 - accuracy: 0.7776 - val_loss: 0.7636 - val_accuracy: 0.7527 - per_epoch_time: 43.1017
Epoch 99/100
157/157 [==============================] - 43s 273ms/step - loss: 0.6765 - accuracy: 0.7777 - val_loss: 0.8135 - val_accuracy: 0.7536 - per_epoch_time: 42.8220
Epoch 100/100
157/157 [==============================] - 43s 271ms/step - loss: 0.6665 - accuracy: 0.7801 - val_loss: 0.7920 - val_accuracy: 0.7468 

### Attention-164

In [ ]:
N_EPOCH = 100
LR = 1e-3

In [14]:
ran_model = Attention164(input_shape=INPUT_SHAPE, num_class=NUM_CLASS)
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention164"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_759 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_582 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_582 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_39 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_180 (Residual  multiple                

In [ ]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/4-a192-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/4-a192-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=0.001)

In [17]:
a164_model = ran_model

a164_model.compile(optimizer=our_optimizer, 
                   loss=tf.keras.losses.CategoricalCrossentropy(), 
                   metrics=['accuracy'])


history = a164_model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 98s 425ms/step - loss: 2.1079 - accuracy: 0.1941 - val_loss: 2.1368 - val_accuracy: 0.1748
Epoch 2/10
157/157 [==============================] - 62s 395ms/step - loss: 1.9617 - accuracy: 0.2410 - val_loss: 2.9659 - val_accuracy: 0.1587
Epoch 3/10
157/157 [==============================] - 62s 395ms/step - loss: 1.9016 - accuracy: 0.2630 - val_loss: 3.4470 - val_accuracy: 0.1594


KeyboardInterrupt: ignored